In [1]:
import numpy as np
import workload
import optimize

In [21]:
R = workload.AllRange(64)
ans = optimize.augmented_optimization(R.WtW, np.random.rand(4, 64))
B = np.reshape(ans['res'].x, (4,64))
I = np.eye(64)
init = np.vstack([np.kron(I, B), np.kron(B,I), np.kron(B,B)])

In [22]:
R = workload.Range2D(64)
ans1 = optimize.augmented_optimization(R.WtW, np.random.rand(*init.shape))
ans2 = optimize.augmented_optimization(R.WtW, init)

print ans1['res'].fun
print ans2['res'].fun

927628148.371
1029913493.36


In [24]:
print ans1['time']
print ans2['time']

1071.43761802
9.80535793304


In [43]:
import autograd.numpy as np
from autograd import grad, primitive

In [67]:
R = workload.AllRange(64)
WtW = R.WtW
@primitive
def expected_error(A):
    AtA = np.dot(A.T, A)
    AtA1 = np.linalg.pinv(AtA)
    X = np.dot(AtA1, WtW)
    return np.trace(X)

def make_grad_expected_error(g, ans, vs, gvs, A):
    AtA = np.dot(A.T, A)
    AtA1 = np.linalg.pinv(AtA)
    X = np.dot(AtA1, WtW)
    Y = np.dot(X, AtA1)
    return -2.0*np.dot(A, Y)

expected_error.defvjp(make_grad_expected_error)

In [74]:
def parameterization(theta):
    A = np.vstack([np.eye(64), np.reshape(theta, (4,64))])
    return A / A.sum(axis=0)

def foo(theta):
    return expected_error(parameterization(theta))
    
#A = np.random.rand(64,64)
#print expected_error(A)
#print grad(expected_error)(A)
theta = np.random.rand(4*64)
print foo(theta)
print grad(foo)(theta)

222778.826289
[  2.86137528e+03   1.18033446e+03   7.22749039e+02   2.02788891e+03
   2.49308321e+03   2.13275271e+03   1.99873529e+03   1.60257027e+03
   2.00901955e+03   1.86287832e+03   2.75062599e+03   2.18814932e+03
   2.88006106e+03   2.65630476e+03   2.01803901e+03   3.49028738e+03
   3.73321542e+03   1.38776382e+03   3.92195525e+03   3.43252382e+03
   3.62654119e+03   4.69984427e+03   2.15468695e+03   3.11412162e+03
   2.92725034e+03   4.39463733e+03   2.66070988e+03   3.29663701e+03
   2.87519391e+03   2.76611997e+03   3.50676085e+03   2.24918905e+03
   3.18282638e+03   1.99410686e+03   2.01231054e+03   1.75751055e+03
   1.85302289e+03   1.67261466e+03   1.60663701e+03   1.32022055e+03
   1.31877459e+03   1.19381028e+03   1.53548144e+03   1.14254193e+03
   1.41131636e+03   1.44721162e+03   1.74079349e+03   8.74670598e+02
   1.24789872e+03   1.23790731e+03   1.10512050e+03   1.47399595e+03
   1.52039616e+03   1.22601211e+03   1.08719967e+03   2.03375013e+03
   1.39795119e+03   

In [82]:
W = R.W
eps = np.random.laplace(0, 1.0, size=A.shape[0])
def err(A):
    A1 = np.linalg.inv(A)
    e = np.dot(W, np.dot(A1, eps))
    return np.dot(e, e)

A = np.random.rand(64,64)
err(A)
#print grad(err)(A)

373623.35303084709